In [1]:
import pandas as pd
from pathlib import Path

In [2]:
BASE_DIR = Path('__file__').resolve().parent  # project root
DATA_DIR = BASE_DIR / "data"
DATA_DIR

WindowsPath('C:/Users/kritr/Downloads/INTERNSHIP/Everlytics_Assignment/data')

In [3]:
paths = [
  f"{DATA_DIR}/products.csv",
  f"{DATA_DIR}/inventory.csv",
  f"{DATA_DIR}/order_20251025.csv",
  f"{DATA_DIR}/order_20251026.csv",
]
for p in paths:
    print("=== FILE:", p)
    df = pd.read_csv(p)
    print(df.shape)
    print(df.dtypes)
    print(df.head(5).to_string(index=False))
    print("null counts:")
    print(df.isna().sum())
    print("\n")

=== FILE: C:\Users\kritr\Downloads\INTERNSHIP\Everlytics_Assignment\data/products.csv
(4, 4)
product_id        int64
product_name     object
category         object
price           float64
dtype: object
 product_id                product_name    category  price
       1001                 Classic Tee     Apparel  19.99
       1002               Running Shoes    Footwear  79.99
       1003                  Coffee Mug        Home   9.50
       1004 Noise-Cancelling Headphones Electronics 199.00
null counts:
product_id      0
product_name    0
category        0
price           0
dtype: int64


=== FILE: C:\Users\kritr\Downloads\INTERNSHIP\Everlytics_Assignment\data/inventory.csv
(4, 4)
product_id            int64
warehouse_id         object
stock_on_hand         int64
last_restock_date    object
dtype: object
 product_id warehouse_id  stock_on_hand last_restock_date
       1001           W1            120        2025-10-18
       1002           W2             40        2025-10-20
       1

In [4]:
def parse_date_column(df, column_name, date_format="%Y-%m-%d"):
    """
    Safely parse a date column into datetime64[ns].
    
    Parameters
    ----------
    df : pandas.DataFrame
        Input dataframe.
    column_name : str
        Name of the column containing date strings.
    date_format : str or None
        If known, provide exact date format for faster + stricter parsing.
        If None, pandas will infer format (slower but flexible).

    Returns
    -------
    good_df : pandas.DataFrame
        Rows with successfully parsed dates.
    bad_df : pandas.DataFrame
        Rows where parsing failed (NaT), with '_parse_error' column.
    """

    df = df.copy()

    # Step 1: strict parsing using provided format
    parsed = pd.to_datetime(df[column_name], format=date_format, errors="coerce")

    # Step 2: Identify bad rows
    bad_mask = parsed.isna()

    bad_df = df[bad_mask].copy()
    if not bad_df.empty:
        bad_df["_parse_error"] = f"Failed to parse dates in column '{column_name}' using format '{date_format}'"

    # Step 3: good rows
    good_df = df[~bad_mask].copy()
    good_df[column_name] = parsed[~bad_mask]

    # Step 4: OPTIONAL fallback (only for bad rows)
    # Try flexible parsing automatically if needed
    if not bad_df.empty:
        fallback_parsed = pd.to_datetime(bad_df[column_name], errors="coerce")
        still_bad = fallback_parsed.isna()

        # good fallback rows
        fallback_good = bad_df[~still_bad].copy()
        fallback_good[column_name] = fallback_parsed[~still_bad]

        # truly bad rows
        final_bad = bad_df[still_bad].copy()
        final_bad["_parse_error"] = "Flexible parsing also failed."

        # merge good + fallback good
        good_df = pd.concat([good_df, fallback_good], ignore_index=True)
        bad_df = final_bad

    return good_df, bad_df

In [7]:
df_orders25 = pd.read_csv(DATA_DIR / "order_20251025.csv")
df_orders26 = pd.read_csv(DATA_DIR/"order_20251026.csv")
df_inventory = pd.read_csv(DATA_DIR/"inventory.csv")
df_product = pd.read_csv(DATA_DIR/"products.csv")

In [8]:
df_orders26, bad = parse_date_column(df_orders26, "order_date")

print("Parsed types:")
print(df_orders26.dtypes)
print("\nBad rows (if any):")
print(bad)

Parsed types:
order_id                 int64
order_date      datetime64[ns]
product_id               int64
qty                      int64
unit_price             float64
user_id                  int64
order_status            object
dtype: object

Bad rows (if any):
Empty DataFrame
Columns: [order_id, order_date, product_id, qty, unit_price, user_id, order_status]
Index: []


In [10]:
df_orders25, bad = parse_date_column(df_orders25, "order_date")

print("Parsed types:")
print(df_orders25.dtypes)
print("\nBad rows (if any):")
print(bad)

Parsed types:
order_id                 int64
order_date      datetime64[ns]
product_id               int64
qty                      int64
unit_price             float64
user_id                  int64
order_status            object
dtype: object

Bad rows (if any):
Empty DataFrame
Columns: [order_id, order_date, product_id, qty, unit_price, user_id, order_status]
Index: []


In [11]:
df_inventory, bad = parse_date_column(df_inventory, "last_restock_date")

print("Parsed types:")
print(df_inventory.dtypes)
print("\nBad rows (if any):")
print(bad)

Parsed types:
product_id                    int64
warehouse_id                 object
stock_on_hand                 int64
last_restock_date    datetime64[ns]
dtype: object

Bad rows (if any):
Empty DataFrame
Columns: [product_id, warehouse_id, stock_on_hand, last_restock_date]
Index: []


In [12]:
df_orders25.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3 entries, 0 to 2
Data columns (total 7 columns):
 #   Column        Non-Null Count  Dtype         
---  ------        --------------  -----         
 0   order_id      3 non-null      int64         
 1   order_date    3 non-null      datetime64[ns]
 2   product_id    3 non-null      int64         
 3   qty           3 non-null      int64         
 4   unit_price    3 non-null      float64       
 5   user_id       3 non-null      int64         
 6   order_status  3 non-null      object        
dtypes: datetime64[ns](1), float64(1), int64(4), object(1)
memory usage: 300.0+ bytes


In [13]:
df_orders26.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2 entries, 0 to 1
Data columns (total 7 columns):
 #   Column        Non-Null Count  Dtype         
---  ------        --------------  -----         
 0   order_id      2 non-null      int64         
 1   order_date    2 non-null      datetime64[ns]
 2   product_id    2 non-null      int64         
 3   qty           2 non-null      int64         
 4   unit_price    2 non-null      float64       
 5   user_id       2 non-null      int64         
 6   order_status  2 non-null      object        
dtypes: datetime64[ns](1), float64(1), int64(4), object(1)
memory usage: 244.0+ bytes


In [14]:
df_inventory.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4 entries, 0 to 3
Data columns (total 4 columns):
 #   Column             Non-Null Count  Dtype         
---  ------             --------------  -----         
 0   product_id         4 non-null      int64         
 1   warehouse_id       4 non-null      object        
 2   stock_on_hand      4 non-null      int64         
 3   last_restock_date  4 non-null      datetime64[ns]
dtypes: datetime64[ns](1), int64(2), object(1)
memory usage: 260.0+ bytes


In [15]:
df_product.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4 entries, 0 to 3
Data columns (total 4 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   product_id    4 non-null      int64  
 1   product_name  4 non-null      object 
 2   category      4 non-null      object 
 3   price         4 non-null      float64
dtypes: float64(1), int64(1), object(2)
memory usage: 260.0+ bytes


In [21]:
from quickshop_etl.readers import resolve_data_dir, read_orders_for_date, read_inventory, read_products
from quickshop_etl.validation import validate_dataframe, OrderSchema

from quickshop_etl.transforms import add_order_total, enrich_with_products, add_order_date_iso, compute_daily_category_revenue
from quickshop_etl.writers import write_parquet_partition, write_summary_json, write_bad_rows_csv

In [17]:
DATA_DIR = resolve_data_dir()               # should point to your data folder
print("DATA_DIR:", DATA_DIR)

DATA_DIR: C:\Users\kritr\Downloads\INTERNSHIP\Everlytics_Assignment\data


In [18]:
orders_good, orders_bad = read_orders_for_date("20251025", data_dir=None)  # module will auto-detect
print("orders_good dtypes:")
print(orders_good.dtypes)
print("orders_bad rows:", len(orders_bad))

orders_good dtypes:
order_id                 int64
order_date      datetime64[ns]
product_id               int64
qty                      int64
unit_price             float64
user_id                  int64
order_status            object
dtype: object
orders_bad rows: 0


In [19]:
good_validated, bad_validated = validate_dataframe(orders_good, OrderSchema)
print("validated good rows:", len(good_validated), "validated bad rows:", len(bad_validated))
if len(bad_validated):
    print(bad_validated.head())

validated good rows: 3 validated bad rows: 0


In [28]:
df_25 = add_order_total(df_orders25)
enriched_25 = enrich_with_products(df_25, df_product)
enriched_25 = add_order_date_iso(enriched_25)
print(enriched_25[['order_id','order_total','product_name','order_date_iso']])
print(compute_daily_category_revenue(enriched_25))

   order_id  order_total                 product_name order_date_iso
0     50001        39.98                  Classic Tee     2025-10-25
1     50002         9.50                   Coffee Mug     2025-10-25
2     50003       199.00  Noise-Cancelling Headphones     2025-10-25
  order_date_iso     category  total_revenue  total_units
0     2025-10-25      Apparel          39.98            2
1     2025-10-25  Electronics         199.00            1
2     2025-10-25         Home           9.50            1


In [29]:
p = write_parquet_partition(enriched_25, partition_value=enriched_25['order_date_iso'].iloc[0], output_dir="output/processed")
summary_25 = {"date": enriched_25['order_date_iso'].iloc[0], "rows": len(enriched_25), "revenue": float(enriched_25['order_total'].sum())}
write_summary_json(summary_25, output_dir="ouSAtput/summaries")
write_bad_rows_csv(orders_bad, name=f"orders-{"20251025"}", output_dir="output/bad_rows")

In [30]:
df_26 = add_order_total(df_orders26)
enriched_26 = enrich_with_products(df_26, df_product)
enriched_26 = add_order_date_iso(enriched_26)
print(enriched_26[['order_id','order_total','product_name','order_date_iso']])
print(compute_daily_category_revenue(enriched_26))

   order_id  order_total   product_name order_date_iso
0     50004        79.99  Running Shoes     2025-10-26
1     50005        59.97    Classic Tee     2025-10-26
  order_date_iso  category  total_revenue  total_units
0     2025-10-26   Apparel          59.97            3
1     2025-10-26  Footwear          79.99            1


In [31]:
p = write_parquet_partition(enriched_26, partition_value=enriched_26['order_date_iso'].iloc[0], output_dir="output/processed")
summary_26 = {"date": enriched_26['order_date_iso'].iloc[0], "rows": len(enriched_26), "revenue": float(enriched_26['order_total'].sum())}
write_summary_json(summary_26, output_dir="output/summaries")
write_bad_rows_csv(orders_bad, name=f"orders-{"20251026"}", output_dir="output/bad_rows")